In [1]:
import os
import h5py

# import matplotlib.pyplot as plt
import time, pickle, pandas

import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import layers

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import ssl
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ssl._create_default_https_context = ssl._create_unverified_context


class LFWLoader:
    """
    Params:
        data_home: path to store LFW data
        target_names (ndarray): list of target names
    """

    def __init__(self, data_home='~/Downloads/scikit_learn_data'):
        """
        Args:
             data_home (str, optional): Default to '~/Downloads/scikit_learn_data'
        """
        self.data_home = data_home

    def load_lfw_data(self,
                      funneled=True,
                      resize=1,
                      min_faces_per_person=2,
                      test_size=0.25,
                      random_state=42):
        """
        Args:
            funneled (boolean, optional): Default to True. Download the funneled dataset.
            resize (float, optional): Default to 1. Ratio to resize the face pictures.
            min_faces_per_person (int, optional): Default to 2. The extracted dataset will only retain pictures of people that have at least min_faces_per_person different pictures.
            test_size (float, int, optional): Default to 0.25. If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples.
            random_state (int, RandomState instance, optional): Default to 42. If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator.
        Returns:
            X_train (ndarray)
            X_test (ndarray)
            y_train (ndarray): onehot label
            y_test (ndarray): onehot label
        """
        # #############################################################################
        # Download the data, if not already on disk and load it as numpy arrays

        lfw_people = fetch_lfw_people(
            data_home=self.data_home,
            funneled=funneled,
            resize=resize,
            min_faces_per_person=min_faces_per_person,
            color=True
            )

        # introspect the images arrays to find the shapes (for plotting)
        n_samples, d, h, w = lfw_people.images.shape

        # for machine learning we use the 2 data directly (as relative pixel
        # positions info is ignored by this model)
        X = lfw_people.images
        n_features = X.shape[1]

        # the label to predict is the id of the person
        y = lfw_people.target
        y = OneHotEncoder(sparse=False).fit_transform(np.reshape(y, (len(y), 1)))

        self.target_names = lfw_people.target_names # store the list of target names
        n_classes = self.target_names.shape[0]

        print("Total dataset size:")
        print("n_samples: %d" % n_samples)
        print("n_features: %d" % n_features)
        print("n_classes: %d" % n_classes)
        # print("image size: " + str(h) + ", " + str(w))

        # #############################################################################
        # Split into a training set and a test set using a stratified k fold

        # split into a training and testing set
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state)

        print('Done!')
        return X_train, X_test, y_train, y_test

In [3]:
batch_size = 16
nb_train = 14513
nb_test = 1791
img_width, img_height = 250, 250
train_data_dir = '../CACD2000/train'
validation_data_dir = '../CACD2000/val'


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=nb_train,
        shuffle=True)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=nb_test)

Found 14513 images belonging to 200 classes.
Found 1791 images belonging to 200 classes.


In [4]:
# import os, os.path
# nb_train_samples = len([name for name in os.listdir(train_data_dir) if os.path.isfile(name)])
# nb_validation_samples = len([name for name in os.listdir(validation_data_dir) if os.path.isfile(name)])

# steps_per_epoch_train = nb_train_samples / batch_size
# steps_per_epoch_val = nb_validation_samples / batch_size

# nc = len(train_generator.class_indices)

In [5]:
tensorboard_callback = TensorBoard(log_dir='./logs/cacd2000', histogram_freq=0, write_graph=True, write_images=False)
checkpoint = ModelCheckpoint("./models/cacd2000.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [6]:
# # from keras.models import Model

# eva_data_dir = './data/c_test'

# eva_datagen = ImageDataGenerator(rescale=1./255)

# eva_generator = eva_datagen.flow_from_directory(
#         eva_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size)

# tf_model.evaluate_generator(eva_generator)

### Resnet

In [7]:
def res_block(input_tensor, kernel_size, filters, strides=(1, 1)):
    """
    Construct a residual block
                        
                        |------------------------>
    1 x 1 conv, filter1      |
                        |                        |
    kernel_size x kernel_size conv, filter2      |
                        |<-----------------------                         |
    
    Args:
        input_tensor (Tensor): input data
        kernel_size (int): size of 2nd layers of conv2d
        filters (tuple of int): 1 * 3 filter value of 1st conv and 2nd conv
    
    Reference: 
        https://blog.waya.ai/deep-residual-learning-9610bb62c355
        https://blog.csdn.net/wspba/article/details/56019373
        https://blog.csdn.net/qq_25491201/article/details/78405549
    """
#     shortcut = input_tensor
    filter1, filter2 = filters
#     x = Conv2D(filter1, strides = strides, kernel_size=(1, 1), padding='same')(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
    
    x = Conv2D(filter1, kernel_size=kernel_size, strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filter2, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
#     x = Conv2D(filter3, kernel_size=(1, 1), padding='same')(x)
#     x = BatchNormalization()(x)
    
    shortcut = Conv2D(filter2, kernel_size=(1, 1), strides=strides, padding='same')(input_tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    
    return x
    
              

In [8]:
def build_resnet(framework='tf', input_shape=(3, 250, 250), nc=200):
    """
    """

    if framework == 'th':
        # build the VGG16 network in Theano weight ordering mode
        backend.set_image_dim_ordering('th')
    else:
        # build the VGG16 network in Tensorflow weight ordering mode
        backend.set_image_dim_ordering('tf')
    
    input_tensor = Input(shape=input_shape)
    x = input_tensor

    # residual units
    
    for i in range(3):
        x = res_block(x, 3, (64, 64))
    
    for i in range(4):
        x = res_block(x, 3, (128, 128))
        
    for i in range(4):
        x = res_block(x, 3, (256, 256))
       
    for i in range(3):
        x = res_block(x, 3, (512, 512))
    
    x = layers.GlobalAveragePooling2D()(x)
    
    x = Dense(units=nc, kernel_initializer="he_normal",
                      activation="softmax")(x)
    
    model = Model(input_tensor, x)
    return model

In [9]:
# X_train, X_test, y_train, y_test = LFWLoader().load_lfw_data()
X_train, y_train = train_generator.next()
X_test, y_test = test_generator.next()

In [10]:
print('Total dataset size')
print('train data size = ' + str(X_train.shape))
print('test data size = ' + str(y_test.shape))


Total dataset size
train data size = (14513, 250, 250, 3)
test data size = (1791, 200)


In [11]:
# batch_size = 32
# train_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# train_datagen.fit(X_train)
# test_datagen.fit(X_test)

# train_datagenerator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
# test_datagenerator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [12]:
input_shape = X_train.shape[1:]
model = build_resnet(input_shape=input_shape, nc=200) # tensorflow dim ordering is like this
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250, 250, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 250, 250, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 250, 250, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 250, 250, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          batch_size=1, 
          epochs=10, 
          verbose=1, 
          validation_data = (X_test, y_test),
          callbacks=[tensorboard_callback, checkpoint],
          shuffle=True)

Train on 14513 samples, validate on 1791 samples
Epoch 1/10
13322/14513 [==========================>...] - ETA: 1:15:22 - loss: 5.2914 - acc: 0.0050

In [ ]:
# model.fit_generator(train_generator, 
#               initial_epoch=0, 
#               verbose=1, 
#               validation_data=validation_generator, 
#               steps_per_epoch=steps_per_epoch_train, 
#               epochs=10, 
#               callbacks=[tensorboard_callback, checkpoint])